In [ ]:
0#Importing Packages

#Data
import pandas as pd
import numpy as np

#ML Frameworks
#scikit learn - library for predictions
#statsmodels - library for models
import sklearn as sk
import statsmodels as sm

#Data Visualization
import seaborn as sns
import matplotlib 


## Understanding Key Drivers of Price

Real estate prices can be seperated effectively in to two distinct categories; **county and metro**. As the location itself acts as a key distinction between the average type, size and price of the property. 

To better understand the market demand, we want to examine the metrics that indicate the prices at which properties are being listed/sold in each particular region. 

The metrics that we want to look at are:
>
 * 'property_type_id': id number indicating which type of property (i.e All Residences, Single Family Homes, Townhouses etc.)
 * 'period_begin': start date
 * 'period_end': end date
 * 'period_duration': duration of the data record
 * 'median_sale_price': median sale price of homes sold
 * 'median_list_price': median list price of homes
 * 'avg_sale_to_list': ratio comparing the sale price and list price
 * 'last_updated' : retreiving the latest updated data
 * 'average annual inflation' : an important factor when the comes to the valuation of USD when purchasing property.



In [ ]:
#Data Wrangling from Redfin
url2 = 'https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/us_national_market_tracker.tsv000.gz'
national_df = pd.read_csv(url2, sep = "\t")


In [ ]:
national_df.columns

Index(['period_begin', 'period_end', 'period_duration', 'region_type',
       'region_type_id', 'table_id', 'is_seasonally_adjusted', 'region',
       'city', 'state', 'state_code', 'property_type', 'property_type_id',
       'median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy',
       'median_list_price', 'median_list_price_mom', 'median_list_price_yoy',
       'median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf',
       'median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold',
       'homes_sold_mom', 'homes_sold_yoy', 'pending_sales',
       'pending_sales_mom', 'pending_sales_yoy', 'new_listings',
       'new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom',
       'inventory_yoy', 'months_of_supply', 'months_of_supply_mom',
       'months_of_supply_yoy', 'median_dom', 'median_dom_mom',
       'median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom',
       'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom',
 

In [ ]:
national_df = national_df[['property_type_id','period_begin', 'period_end', 'period_duration' ,'median_list_price','median_sale_price','avg_sale_to_list', 'last_updated']]

In [ ]:
# property_type_id == -1; All residential homes
national_df = national_df.loc[national_df['property_type_id'] == -1]

#Removing Duplicate Entries, Leaving Latest Update
national_df = national_df.loc[national_df['last_updated'] == '2022-04-10 14:49:10']

#Sorting by ascending order of entry period end date
national_df.sort_values('period_end', inplace = True)


In [ ]:
#Inflation Data
url3 = 'https://raw.githubusercontent.com/ayiply/test_repo/main/inflation_data.csv'
inflation_df = pd.read_csv(url3)

inflation_df.columns
inflation_df = inflation_df.rename({'inflation rate' : 'inf_rate'}, axis = 'columns' )

inf_dict = dict(zip(inflation_df.year, inflation_df.inf_rate))


In [ ]:
#List for inflation rates by year in period_begin
inflation_column = []

#converting period_begins column into iterable list
ser = pd.DataFrame(national_df, columns = ['period_begin'])
dates = ser.squeeze()
date_list = dates.to_list()

#assigning annual inflation rate based on year found in list of dates' strings
for key in inf_dict:
  for entry in date_list:
    if str(key) in entry:
      inflation_column.append(inf_dict[key])
  
#appending column to national_df
national_df['inflation_by_year'] = inflation_column

In [ ]:
national_df

,property_type_id,period_begin,period_end,period_duration,median_list_price,median_sale_price,avg_sale_to_list,last_updated,inflation_by_year
1028,-1,2012-01-01,2012-01-31,30,182741.0,158955.0,0.956285,2022-04-10 14:49:10,0.02
1016,-1,2012-02-01,2012-02-29,30,192252.0,159915.0,0.958041,2022-04-10 14:49:10,0.02
337,-1,2012-03-01,2012-03-31,30,198860.0,170681.0,0.961785,2022-04-10 14:49:10,0.02
1080,-1,2012-04-01,2012-04-30,30,198927.0,177774.0,0.965150,2022-04-10 14:49:10,0.02
539,-1,2012-05-01,2012-05-31,30,200718.0,186050.0,0.968565,2022-04-10 14:49:10,0.02
...,...,...,...,...,...,...,...,...,...
1014,-1,2021-11-01,2021-11-30,30,371843.0,382460.0,1.006288,2022-04-10 14:49:10,0.05
946,-1,2021-12-01,2021-12-31,30,366811.0,381527.0,1.004854,2022-04-10 14:49:10,0.05
808,-1,2022-01-01,2022-01-31,30,395800.0,376519.0,1.003992,2022-04-10 14:49:10,0.07
1057,-1,2022-02-01,2022-02-28,30,415025.0,388527.0,1.012548,2022-04-10 14:49:10,0.07


# **Notes for the team:** 
So the national_df DataFrame contains aggregate values for all residential homes across the US. Potentially if we wanted to look at specific states (like California, Washington, etc) there are more data for location specific metrics which we can further investigate. 